# Racetrack with SB3's SAC

##  Warming up
We start with a few useful installs and imports:

In [ ]:
# Installs

# Install environment and agent
!pip install highway-env
# TODO: we use the bleeding edge version because the current stable version does not support the latest gym>=0.21 versions. Revert back to stable at the next SB3 release.
!pip install git+https://github.com/DLR-RM/stable-baselines3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 24.3 MB/s eta 0:00:00
  Cloning https://github.com/DLR-RM/stable-baselines3 to /tmp/pip-req-build-pv70rdgn
  Running command git clone --filter=blob:none --quiet https://github.com/DLR-RM/stable-baselines3 /tmp/pip-req-build-pv70rdgn
  Resolved https://github.com/DLR-RM/stable-baselines3 to commit 5623d98f9d6bcfd2ab450e850c3f7b090aef5642
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12

In [ ]:
# Environment
import gymnasium as gym
import highway_env

# Agent
from stable_baselines3 import SAC

In [ ]:

# Visualization utils
import sys
from tqdm.notebook import trange
!pip install gym pyvirtualdisplay
!apt-get install -y xvfb ffmpeg
!git clone https://github.com/Farama-Foundation/HighwayEnv.git 2> /dev/null
sys.path.insert(0, '/content/HighwayEnv/scripts/')
from utils import record_videos, show_videos

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.10).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


## Training
Run tensorboard locally to visualize training.

In [ ]:
model = SAC('MlpPolicy', 'racetrack-v0',verbose=2)
model.learn(int(2e5))

Using cpu device
Creating environment from the given name 'racetrack-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.5e+03  |
|    ep_rew_mean     | 17       |
| time/              |          |
|    episodes        | 4        |
|    fps             | 14       |
|    time_elapsed    | 427      |
|    total_timesteps | 6004     |
| train/             |          |
|    actor_loss      | -7.52    |
|    critic_loss     | 0.000622 |
|    ent_coef        | 0.17     |
|    ent_coef_loss   | -2.97    |
|    learning_rate   | 0.0003   |
|    n_updates       | 5903     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.02e+03 |
|    ep_rew_mean     | 58       |
| time/              |          |
|    episodes        | 8        |
|    fps             | 14       |
|    time_elapsed    | 581      |


KeyboardInterrupt: 

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive',force_remount=True)
path = r"/content/gdrive/MyDrive/Colab Notebooks/"

Mounted at /content/gdrive


In [ ]:
# save the model
model.save(path + "sac_racetrack2e5")

In [ ]:
# load the model
model = SAC.load(path + "sac_racetrack")

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: Can't get attribute '_function_setstate' on <module 'cloudpickle.cloudpickle' from '/usr/local/lib/python3.10/dist-packages/cloudpickle/cloudpickle.py'>
  warnings.warn(


## Testing

Visualize a few episodes

In [ ]:
env = gym.make('racetrack-v0', render_mode='rgb_array')
env = record_videos(env)
for episode in trange(3, desc='Test episodes'):
    (obs, info), done = env.reset(), False
    for i in range(100):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
        if done:
          break
env.close()
show_videos()

/usr/local/lib/python3.10/dist-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /content/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Test episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Moviepy - Building video /content/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-0.mp4
